# <center>Título da Proposta</center>
---
 
Objeto:

Desenvolvido por:
* <b>George Ulguim Pedra</b> - gupbadabum@gmail.com

Criado:     01-03-2023

Atualizado: 10-05-2023

In [1]:
rm(list = ls())

In [2]:
library("rgdal")

Loading required package: sp

rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.3.3, released 2021/10/25
Path to GDAL shared files: /home/hellgate/anaconda3/envs/r-env3/share/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 8.1.1, September 1st, 2021, [PJ_VERSION: 811]
Path to PROJ shared files: /home/hellgate/anaconda3/envs/r-env3/share/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.



## Leitura de dados

### Planilhas de casos e dados climáticos

In [3]:
cas_lta <- read.csv("../2-Input/1-LTA/LTA_incidencia_2001_2005_wide.csv", header = T, sep = ";", dec = ",")
cli_ind <- read.csv("../2-Input/1-LTA/Ind_Climaticos_Simples_LTA_03_12_2023.csv", header = T, sep = ";", dec = ",")
cli_bru <- read.csv("../2-Input/1-LTA/Dados_Climaticos_Anuais_LTA.csv", header = T, sep = ";", dec = ".")
cli_nor <- read.csv("../2-Input/1-LTA/Dados_Climaticos_Anuais_NORM_LTA.csv", header = T, sep = ";", dec = ".")

### Carregando Shapefiles

In [4]:
map0 <- readOGR("/media/hellgate/16D7D0A761E2DB6A/MEGA/Shapefiles/Br_2019/br_municipios/BR_Municipios_2019.shp")

Warning message in OGRSpatialRef(dsn, layer, morphFromESRI = morphFromESRI, dumpSRS = dumpSRS, :
“Discarded datum Sistema_de_Referencia_Geocentrico_para_las_AmericaS_2000 in Proj4 definition: +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs”


OGR data source with driver: ESRI Shapefile 
Source: "/media/hellgate/16D7D0A761E2DB6A/MEGA/Shapefiles/Br_2019/br_municipios/BR_Municipios_2019.shp", layer: "BR_Municipios_2019"
with 5572 features
It has 4 fields


In [5]:
head(cas_lta)

,ibge_code7,ibge_code6,uf,pop_ibge_2019,X2001,X2002,X2003,X2004,X2005,Total,incidence,indicence_100k
,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,1100015,110001,RO,22945,157,107,105,106,48,523,0.022793637,2.279364e-07
2,1100023,110002,RO,107863,160,154,124,185,115,738,0.006842013,6.842013e-08
3,1100031,110003,RO,5312,4,5,11,23,5,48,0.009036145,9.036145e-08
4,1100049,110004,RO,85359,139,111,112,142,152,656,0.007685188,7.685188e-08
5,1100056,110005,RO,16323,3,15,14,6,16,54,0.003308215,3.308215e-08
6,1100064,110006,RO,15882,24,36,39,29,26,154,0.009696512,9.696512e-08


In [6]:
head(cli_ind)

,CD_GEOCMU,NM_MUNICIP,X51022.2020,X51022.2030.O,X51022.2050.O,X51022.2030.P,X51022.2050.P,X51023.2020,X51023.2030.O,X51023.2050.O,X51023.2030.P,X51023.2050.P,X51024.2020,X51024.2030.O,X51024.2050.O,X51024.2030.P,X51024.2050.P
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1100015,Alta Floresta D'Oeste,0.87,0.95,1.00,0.97,1.00,0.39,0.43,0.44,0.40,0.43,0.50,0.46,0.42,0.44,0.40
2,1100023,Ariquemes,0.87,0.96,1.00,0.98,1.00,0.43,0.44,0.44,0.41,0.42,0.50,0.45,0.41,0.43,0.39
3,1100049,Cacoal,0.86,0.95,1.00,0.97,1.00,0.40,0.42,0.43,0.39,0.40,0.41,0.38,0.33,0.35,0.31
4,1100056,Cerejeiras,0.86,0.94,0.99,0.97,1.00,0.36,0.40,0.40,0.35,0.38,0.44,0.41,0.37,0.38,0.35
5,1100064,Colorado do Oeste,0.80,0.88,0.94,0.91,0.99,0.42,0.46,0.47,0.42,0.44,0.46,0.42,0.38,0.39,0.36
6,1100080,Costa Marques,0.87,0.96,1.00,0.97,1.00,0.38,0.42,0.43,0.38,0.40,0.45,0.41,0.38,0.39,0.36


In [7]:
LTA <- as.numeric(c(cas_lta[, 5], cas_lta[, 6], cas_lta[, 7], cas_lta[, 8], cas_lta[, 9]))

# Contagem de municipios com informações válidas em cada ano da amostra
# excluindo valores UNDEF (NA) e zeros

LTA01 <- length(cas_lta[, 5]) - length(which(cas_lta[, 5] == 0 | is.na(cas_lta[, 5]))) # Ano de 2001
LTA02 <- length(cas_lta[, 6]) - length(which(cas_lta[, 6] == 0 | is.na(cas_lta[, 6]))) # Ano de 2002
LTA03 <- length(cas_lta[, 7]) - length(which(cas_lta[, 7] == 0 | is.na(cas_lta[, 7]))) # Ano de 2003
LTA04 <- length(cas_lta[, 8]) - length(which(cas_lta[, 8] == 0 | is.na(cas_lta[, 8]))) # Ano de 2004
LTA05 <- length(cas_lta[, 9]) - length(which(cas_lta[, 9] == 0 | is.na(cas_lta[, 9]))) # Ano de 2005

CLTA <- matrix(NA, ncol = 5, nrow = 1)
dimnames(CLTA)[[1]] <- "Casos"
dimnames(CLTA)[[2]] <- c(2001:2005)
CLTA[1, ] <- c(LTA01, LTA02, LTA03, LTA04, LTA05)

CLTA

,2001,2002,2003,2004,2005
Casos,1923,1988,1918,1810,1941


In [8]:
MED <- function(x) {
    Y <- median(x, na.rm = T)
}

LTA.m <- apply(cas_lta[, 5:9], 1, MED)

DADO_INPUT <- data.frame(GEOCOD = cas_lta[, 1], LTA.P = LTA.m, LTA.C = LTA.m)

DADO_INPUT[, 4:18] <- NA

In [9]:
# Inserindo as variaveis climáticas referente ao período histórico e indexando pelo geocódido

for (i in 1:nrow(DADO_INPUT)) {
    pos <- which(DADO_INPUT[i, 1] == cli_ind[, 1])
    DADO_INPUT[i, -c(1:3)] <- cli_ind[pos, -c(1:2)]
}

ERROR: Error in x[[jj]][iseq] <- vjj: substituto tem comprimento zero


In [14]:
i

DADO_INPUT[i, ]

cli_ind[pos, ]

dim(cli_ind)

pos

[1] 4607

,GEOCOD,LTA.P,LTA.C,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4607,4300001,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


CD_GEOCMU,NM_MUNICIP,X51022.2020,X51022.2030.O,X51022.2050.O,X51022.2030.P,X51022.2050.P,X51023.2020,X51023.2030.O,X51023.2050.O,X51023.2030.P,X51023.2050.P,X51024.2020,X51024.2030.O,X51024.2050.O,X51024.2030.P,X51024.2050.P
<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


[1] 5570   17

integer(0)